In [51]:
!pip install stable-baselines3
#instalar librerias para el juego
!sudo apt-get update
!sudo apt-get install -y libsdl2-mixer-dev libsdl2-mixer

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:7 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Fetched 3,917 B in 7s (532 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package libsdl2-mixer


In [52]:
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

## Set up utils

In [53]:
from typing import NamedTuple, Dict, Any
from enum import IntEnum
import numpy as np


class StepResult(NamedTuple):
  observation: np.ndarray
  reward: float
  terminated: bool
  truncated: bool
  info: Dict[str, Any]

class Action(IntEnum):
  DOWN  = 0
  LEFT  = 1
  RIGHT = 2
  UP    = 3
  FIRE  = 4
  POLARITY = 5
  FRAGMENT = 6
  ESCAPE = 7
  PAUSE = 8
  ENTER = 9

class Keys(IntEnum):
    DOWN = 1 << 0
    LEFT = 1 << 1
    RIGHT = 1 << 2
    UP = 1 << 3
    FIRE = 1 << 4
    POLARITY = 1 << 5
    FRAGMENT = 1 << 6
    ESCAPE = 1 << 7
    PAUSE = 1 << 8
    ENTER = 1 << 9

class State(IntEnum):
  FRAME      = 0
  SCORE      = 1
  LIVES      = 2
  POWER      = 3
  POLARITY   = 4
  PLAYER_X   = 5
  PLAYER_Y   = 6
  CHAIN      = 7
  CHAIN_COLOR= 8

  RESERVED_START = 9
  RESERVED_END   = 16

  REWARD = 17

  ENEMIES    = 18
  BULLETS    = 179

In [54]:
#state = obs[:State.CHAIN_COLOR]      # frame .. chain_color
#enemy_bullet_data = obs[State.ENEMIES:]     # e_0_x .. end
#obs = np.concatenate((state, enemy_bullet_data))

In [55]:
class Utils:
  #static
  def remove_invalid_actions(action):
    if(action[7] == 1):
      action[7] = 0
    if(action[8] == 1):
      action[8] = 0
    if(action[Action.RIGHT] == 1 and action[Action.LEFT] == 1):
      action[Action.RIGHT] = 0
      action[Action.LEFT] = 0
    if(action[Action.DOWN] == 1 and action[Action.UP] == 1):
      action[Action.DOWN] = 0
      action[Action.UP] = 0
    if(action[Action.ENTER] == 1):
      action[Action.ENTER] = 0
    return action


  def keys_to_actions(keys_mask):
    # 10 actions total
    actions = [0] * len(Action)

    if keys_mask & Keys.UP:
        actions[Action.UP] = 1
    if keys_mask & Keys.DOWN:
        actions[Action.DOWN] = 1
    if keys_mask & Keys.LEFT:
        actions[Action.LEFT] = 1
    if keys_mask & Keys.RIGHT:
        actions[Action.RIGHT] = 1
    if keys_mask & Keys.FIRE:
        actions[Action.FIRE] = 1
    if keys_mask & Keys.POLARITY:
        actions[Action.POLARITY] = 1
    if keys_mask & Keys.FRAGMENT:
        actions[Action.FRAGMENT] = 1
    if keys_mask & Keys.ESCAPE:
        actions[Action.ESCAPE] = 1
    if keys_mask & Keys.PAUSE:
        actions[Action.PAUSE] = 1
    if keys_mask & Keys.ENTER:
        actions[Action.ENTER] = 1

    return actions

In [56]:
import threading

LOG_FILE = "env_log.txt"
log_lock = threading.Lock()  # ensures threads don’t write at the same time

def log(message: str):
    with log_lock:  # prevent race conditions
        with open(LOG_FILE, "a") as f:
            f.write(message + "\n")
log(f"[NEW CONNECTION]")

In [57]:
import socket
import threading
from queue import Queue
from threading import Event
import random
import time
from enum import StrEnum

class GameControllerTCPServer:

  #_instance = None




  class ServerState(StrEnum):
    INITIAL = "INITIAL"
    LISTENING = "LISTENING"
    CONNECTED = "CONNECTED"


  #def __new__(cls):
    ##this singleton stuff does not seem to work
    #print("__new__")
    #if not cls._instance:
      #cls._instance = super().__new__(cls)
    #return cls._instance

  def __init__(self, inbound_queue: Queue, step_allowed: Event):
    self.HEADER = 64
    self.PORT = 0
    self.SERVER_IP = "127.0.0.1"
    self.ADDRESS = (self.SERVER_IP, self.PORT)
    self.FORMAT = "utf-8"
    self.DISCONNECT_MESSAGE = "!DISCONNECT"

    self.currentState = self.ServerState.INITIAL
    #if getattr(self, "_initialized", False):
      #return

    self.ready = Event()
    self.inbound_queue = inbound_queue
    self.step_allowed = step_allowed
    #self._initialization = True



    #print("create server init")

    #AF_INET = ipv4
    #SOCK_STREAM = TCP
    self.server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
      self.server.bind(self.ADDRESS)
      print(f"server binding succesfull to {self.ADDRESS}")
    except:
      print("Error binding to port")
      raise RuntimeError(f"Failed to bind to {self.ADDRESS}")


    # dictionario IP -> [] : cliente
    self.client_messages = {}
    self.first_addr = None


  def start(self):
    self.server.listen()
    self.ready.set()
    print(f"[Listening] Server is listening on {self.SERVER_IP}")
    self.currentState = self.ServerState.LISTENING
    print("SERVER STATE -> LISTENING")
    print("BOUND TO:", self.server.getsockname())
    while True:
      #print("CALLING accept()")
      conn, addr = self.server.accept()
      #print("ACCEPTED:", addr)
      if not self.first_addr:
        self.first_addr = addr
      self.client_messages[addr] = []
      #log("creating handle_client")
      #print("addr" + str(addr))
      #print(self.ADDRESS)
      #print("create handle_client thread")
      thread = threading.Thread(target=self.handle_client, args=(conn, addr))
      thread.start()
      #print(f"[ACTIV CONNECTIONS] {threading.active_count() - 1}")

  def handle_client(self, conn, addr):
    #log(f"[NEW CONNECTION, in handle_client] {addr} connected")
    #print(f"[NEW CONNECTION, in handle_client] {addr} connected")
    #print("this is handle_client")

    connected = True
    self.currentState = self.ServerState.CONNECTED

    while connected:
      try:
        #print("waiting for message length", flush=True)
        msg_length = conn.recv(self.HEADER).decode(self.FORMAT)
        if(msg_length):
          msg_length = int(msg_length)
          #print("waiting for message")
          msg = conn.recv(msg_length).decode(self.FORMAT)
          #print("msg received")
          if(msg == self.DISCONNECT_MESSAGE):
            print("disconected")
            connected = False
          self.client_messages[addr].append(msg)

          self.inbound_queue.put(msg)
          #print("handle client, wait")
          self.step_allowed.wait()
          self.step_allowed.clear()
          #print("handle client, clear")

          conn.sendall("STEP".encode(self.FORMAT))
          #print("step")
      except Exception as e:
        print("Game close closing thread")
        break
    conn.close()

  def get_message(self, addr):
    if addr:
      return self.client_messages.get(addr, [])
    return self.client_messages.get(self.first_addr, [])

  def send_action(self, action):
      self.client.sendall(str(action).encode())

  def close(self):
      self.client.close()

In [58]:
import subprocess
import time
import os
from queue import Queue
from threading import Event
import threading


class GameController:
    def __init__(self, game_path="./nKaruga", headless=False):
        print("create server controller")

        self.inbound_queue = Queue()
        self.step_allowed = Event()

        self.server = GameControllerTCPServer(self.inbound_queue,
                                              self.step_allowed)

        thread = threading.Thread(target=self.server.start, daemon=True)
        thread.start()

        self.server.ready.wait()

        self.game_path = game_path
        self.headless = headless
        self.process = None
        #self.frame_duration = 1.0 / 60.0

    def start(self):
        port = self.server.server.getsockname()[1]
        args = [self.game_path, "-C"]
        if self.headless:
            print("start headless")
            args.append("-V")
            args.append("-S")
            #args.append("-U")
            args.append(f"-P {port}")
        else:
            print("do not start headless")

        self.stderr_file = open("game_stderr.log", "w")

        my_env = os.environ.copy()
        # Prepend /bundle/lib to LD_LIBRARY_PATH. Use ':' to separate paths.
        # Ensure existing LD_LIBRARY_PATH is included if it exists.
        #if 'LD_LIBRARY_PATH' in my_env:
        #    my_env['LD_LIBRARY_PATH'] = '/content/bundle/lib:' + my_env['LD_LIBRARY_PATH']
        #else:
        #    my_env['LD_LIBRARY_PATH'] = '/content/bundle/lib'

        print(f"Starting game: {' '.join(args)}")
        self.process = subprocess.Popen(
            args,
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=self.stderr_file,
            text=True,
            bufsize=0, # Unbuffered
        )
        # Give it a moment to initialize
        print("game process started")
        time.sleep(2.0)

    def reset(self):
        print("Resetting game...")
        # Terminate existing process if running
        if hasattr(self, "process") and self.process.poll() is None:
            self.process.terminate()
            try:
                self.process.wait(timeout=5)
            except subprocess.TimeoutExpired:
                print("Force killing game process")
                self.process.kill()
                self.process.wait()

        # Close previous stderr file if open
        if hasattr(self, "stderr_file") and not self.stderr_file.closed:
            self.stderr_file.close()

        # Start a fresh game process
        self.start()


    #The point of this function is to get the state of the game
    def step(self, action, frames=1) -> np.ndarray:
        """
        Sends an action and waits for the specified number of frames.
        """
        mask = 0
        for i, bit in enumerate(action):
          if bit:
            mask |= (1 << i)

        if self.process is None or self.process.poll() is not None:
            print("Game is not running.")
            exit(1)

        try:
            msg = None
            obs = None
            # Send the action
            self.process.stdin.write(f"{int(mask)}\n")
            self.process.stdin.flush()

            #print("msg")
            #log("log")
            #if self.inbound_queue.empty():
            #    print("inbound queue empty")
            #    #print(self.server.currentState)
            #    #self.step_allowed.wait()
            #    msg = np.zeros(768, dtype=np.float32)
            #    msg[State.LIVES] = 4
            #    self.step_allowed.set()
            #    #print(msg)
            #    obs = msg
            #else:

            msg = self.inbound_queue.get()
            self.step_allowed.set()

            #print("inbound queue")
            self.step_allowed.set()
            #print("controller step set")
            obs = np.fromstring(msg, sep=",", dtype=np.float32)
            #print(f"obs: {obs}")
            #obs fix
            state = obs[:State.CHAIN_COLOR + 1]      # frame .. chain_color
            enemy_bullet_data = obs[State.ENEMIES:]     # e_0_x .. end
            obs = np.concatenate((state, enemy_bullet_data))


            return obs

        except BrokenPipeError as e:
            print("BrokenPipeError caught")
            print(f"Errno: {e.errno}")        # usually errno.EPIPE
            print(f"Strerror: {e.strerror}")  # human-readable
            print(f"Filename: {e.filename}")  # if present
            print("Game process closed unexpectedly.")
            self.close()
            return None


    def close(self):
        if self.process:
            self.process.terminate()
            self.process.wait()
            print("Game process terminated.")

In [59]:
import gymnasium as gym # Keep this import for Env if needed, or if Env is not fully encapsulated in v0_game_env
from stable_baselines3 import PPO


class Agents:

  def random_agent_process(controller: GameController, env: gym.Env):
    while True:
      #read input
      # controller.getMessage() is not a method for GameController. Need to adjust.
      # For random agent, usually, we don't 'get message' from controller like this.
      # Let's assume input processing is handled by the env or removed for simplicity

      action = env.action_space.sample()
      action = action.tolist()
      print(f"Random Agent: {action}")
      print(f"Random Agent: {Utils.keys_to_actions(0)}")
      # Apply remove_invalid_actions from Utils
      action = Utils.remove_invalid_actions(action)

      # Original logic contained redundant checks for escape, pause, etc.
      # Utils.remove_invalid_actions should handle this.
      # The repeated controller.step with 0 action might be part of the game logic for frames, kept for now.
      controller.step(action)
      controller.step(Utils.keys_to_actions(0))
      controller.step(Utils.keys_to_actions(0))
      controller.step(Utils.keys_to_actions(0))
      controller.step(Utils.keys_to_actions(0))

  def ppo_trained_model(controller: GameController, eval_env: gym.Env, model: PPO):
    (obs, info) = eval_env.reset()
    done = False
    while not done:
      action, _ = model.predict(obs)
      action = Utils.remove_invalid_actions(action)

      # The env.step expects to return obs, reward, terminated, truncated, info
      # The controller.step only returns obs.
      # If the agent is interacting with the env directly, it should be eval_env.step(action)
      # If it's interacting with controller directly, then controller.step(action) and then manual termination check.
      # Assuming it should interact with the eval_env which in turn uses the controller.
      obs, reward, terminated, truncated, info = eval_env.step(action)

      # Original code checked termination based on obs[State.LIVES] == 0 directly
      # This is now handled by eval_env.step and its internal logic.
      if terminated or truncated:
        print("EPISODE TERMINATED OR TRUNCATED")
        done = True


In [60]:
import gymnasium as gym
from gymnasium.envs.registration import register
from gymnasium.utils.env_checker import check_env
from gymnasium import spaces
import numpy as np
from stable_baselines3 import PPO

class GameEnv(gym.Env):
  metadata = {'render_modes': ['human'], 'render_fps': 1}

  def __init__(self, render_mode=None):
    self.reset()
    self.renderMode = render_mode
    if(self.renderMode):
      controller = GameController(headless=False)
    else:
      controller = GameController(headless=True)


    controller.start()
    self.controller = controller
    self.max_steps = 1000
    self.current_step = 0

    # action space
    self.action_space = spaces.MultiBinary(len(Action))

    # observation space
    # TODO - define more precise observation space
    self.observation_space = spaces.Box(
    low=-np.inf,
    high=np.inf,
    shape=(769,),
    dtype=np.float32
    )

  def reset(self, seed=None, options=None):
    print("ENV RESET CALLED")
    super().reset(seed=seed)
    self.current_step = 0
    state = np.zeros(769, dtype=np.float32)

    return state, {}

  #The point of this function is to send an action to the game
  # and return the observation, reward, terminated, truncated, info
  def step(self, action) -> StepResult:
    self.current_step += 1
    #print("ENV STEP CALLED: " + str(self.current_step))
    action = Utils.remove_invalid_actions(action)
    #print("action")

    # get reward and state
    #print("controller step")
    obs = self.controller.step(action)
    if(obs is None):
      print("obs is none")
      self.controller.reset()
      return StepResult(np.zeros(769, dtype=np.float32), 0, True, False, {})



    #print(f"obs: {obs}")

    # terminate if lives = 0
    #print("get reward")
    reward = obs[State.REWARD]
    #print("terminated")
    terminated = obs[State.LIVES] == 0 and self.current_step > 1
    #print("truncated")
    truncated = self.current_step >= self.max_steps
    #print("check terminated")
    if(terminated):
      print("TERMINATED")
      print(obs)
      #reset game
      self.controller.reset()


    if(truncated):
      print("TRUNCATED")
      self.controller.reset()
    info = {}
    return StepResult(obs, reward, terminated, truncated, info)
    #return self.controller.step(action)

  def render(self, mode='human'):
    pass

In [61]:
#import gymnasium as gym
#from stable_baselines3 import PPO
#
#import os
#
#def train():
#  model_dir = "models"
#  log_dir = "logs"
#
#  os.makedirs(model_dir, exist_ok=True)
#  os.makedirs(log_dir, exist_ok=True)
#
#  env = GameEnv()#gym.make("MyGame-v0", render_mode=None)
#  #MlpPolicy define una arquitectura de red capas densas,
#  #Tambien se puede ocupar CnnPolicy para una con capas
#  #convolucionales, especialmente para imagenes
#  model = PPO("MlpPolicy", env, verbose=1, device="cpu", tensorboard_log=log_dir)
#  #model.learn(total_timesteps=100000)
#
#  TIMESTEPS = 50_000
#  MAX_ITERS = 5
#  iters = 0
#
#  print("Training...")
#
#  for iters in range(MAX_ITERS):
#    print(f"Iter {iters}")
#
#    model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False)
#    model.save(f"{model_dir}/model_{iters}")

In [62]:
import os

log_file_path = "game_stderr.log"

if os.path.exists(log_file_path):
    with open(log_file_path, "r") as f:
        log_content = f.read()
    if log_content:
        print("--- Content of game_stderr.log ---")
        print(log_content)
        print("------------------------------------")
    else:
        print(f"The log file '{log_file_path}' exists but is empty.")
else:
    print(f"The log file '{log_file_path}' does not exist.")

The log file 'game_stderr.log' exists but is empty.


In [63]:
!ls

 debug.log     game_data.csv	 logs		 models    sample_data
 env_log.txt   game_stderr.log	'model(1).pth'	 nKaruga


In [64]:
!./nKaruga -V -C -P 0

Flag -V enabled (Headless Mode)
Flag -C enabled
Building game LUTs ...
Done

Connection Failed 
FAILURE


### entrenar

In [65]:
!wget https://github.com/MartinCastroMorales-mcm/entornoRL/raw/refs/heads/master/nKaruga

--2025-12-29 01:53:43--  https://github.com/MartinCastroMorales-mcm/entornoRL/raw/refs/heads/master/nKaruga
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/MartinCastroMorales-mcm/entornoRL/refs/heads/master/nKaruga [following]
--2025-12-29 01:53:43--  https://raw.githubusercontent.com/MartinCastroMorales-mcm/entornoRL/refs/heads/master/nKaruga
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3643256 (3.5M) [application/octet-stream]
Saving to: ‘nKaruga.1’

nKaruga.1           100%[===================>]   3.47M  --.-KB/s    in 0.06s   

2025-12-29 01:53:43 (58.9 MB/s) - ‘nKaruga.1’ saved [3643256

In [66]:
!chmod +x nKaruga

In [67]:
!ldd ./nKaruga | grep SDL

	libSDL2-2.0.so.0 => /lib/x86_64-linux-gnu/libSDL2-2.0.so.0 (0x000078735c373000)


In [68]:
!unzip lib.zip -y

unzip:  cannot find or open lib.zip, lib.zip.zip or lib.zip.ZIP.


In [69]:
import torch as th
from torch import nn
from stable_baselines3 import PPO
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.env_util import make_vec_env

class CustomFlatExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim=256):
        super().__init__(observation_space, features_dim)
        # Assuming observation is a flat vector of 768
        self.net = nn.Sequential(
            nn.Linear(observation_space.shape[0], 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, features_dim),
            nn.ReLU()
        )

    def forward(self, observations: th.Tensor) -> th.Tensor:
        return self.net(observations)

In [70]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import torch
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize


def make_env():
    return GameEnv()


def train():
  #if not torch.cuda.is_available():
  #  raise RuntimeError("No GPU found! Please enable a GPU to run this.")
  model_dir = "models"
  log_dir = "logs"

  os.makedirs(model_dir, exist_ok=True)
  os.makedirs(log_dir, exist_ok=True)

  num_envs = 1  # for example
  envs = DummyVecEnv([make_env for _ in range(num_envs)])

  policy_kwargs = dict(
    features_extractor_class=CustomFlatExtractor,
    features_extractor_kwargs=dict(features_dim=256)
  )
  #MlpPolicy define una arquitectura de red capas densas,
  #Tambien se puede ocupar CnnPolicy para una con capas
  #convolucionales, especialmente para imagenes

  env = VecNormalize(
    envs,
    norm_obs=True,
    norm_reward=True,
    clip_obs=10.0,
    clip_reward=10.0,
    gamma=0.99
)
  model = PPO(
      "MlpPolicy",
      env,
      policy_kwargs=policy_kwargs,
      verbose=1,
      device="cpu",
      tensorboard_log=log_dir)
  #model.learn(total_timesteps=100000

  TIMESTEPS = 50_000
  MAX_ITERS = 5
  iters = 0

  print("Training...")

  for iters in range(MAX_ITERS):
    print(f"Iter {iters}")

    model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False)
    #Se prefiere guardar los pesos del modelo ya que guardar el modelo directamente pueda dar problemas al cambiar de entorno
    model.save(f"{model_dir}/model_{iters}")
    #torch.save(model.)


In [71]:
!wget "https://github.com/MartinCastroMorales-mcm/entornoRL/raw/refs/heads/master/model(1).pth"

--2025-12-29 01:53:44--  https://github.com/MartinCastroMorales-mcm/entornoRL/raw/refs/heads/master/model(1).pth
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/MartinCastroMorales-mcm/entornoRL/refs/heads/master/model(1).pth [following]
--2025-12-29 01:53:45--  https://raw.githubusercontent.com/MartinCastroMorales-mcm/entornoRL/refs/heads/master/model(1).pth
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2642813 (2.5M) [application/octet-stream]
Saving to: ‘model(1).pth.1’

model(1).pth.1      100%[===================>]   2.52M  --.-KB/s    in 0.04s   

2025-12-29 01:53:45 (58.4 MB/s) - ‘model

In [72]:
# Arquitectura del BC
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(769, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 6)
    )
  def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [73]:
class BCFeatureExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim=512, backbone=None):
        super().__init__(observation_space, features_dim)
        # Remove last layer
        self.backbone = nn.Sequential(*list(backbone.linear_relu_stack.children())[:-1])

    def forward(self, x):
        return self.backbone(x)

In [74]:
def train_from_bc():

  model_dir = "models"
  log_dir = "logs"

  os.makedirs(model_dir, exist_ok=True)
  os.makedirs(log_dir, exist_ok=True)

  num_envs = 1  # for example
  envs = DummyVecEnv([make_env for _ in range(num_envs)])

  #MlpPolicy define una arquitectura de red capas densas,
  #Tambien se puede ocupar CnnPolicy para una con capas
  #convolucionales, especialmente para imagenes

  env = VecNormalize(
    envs,
    norm_obs=True,
    norm_reward=True,
    clip_obs=10.0,
    clip_reward=10.0,
    gamma=0.99
  )
  #########################################
  bc_backbone = NeuralNetwork()
  bc_backbone.load_state_dict(torch.load("model(1).pth"))


  for param in bc_backbone.parameters():
    param.requires_grad = False

  policy_kwargs = dict(
    features_extractor_class=BCFeatureExtractor,
    features_extractor_kwargs=dict(features_dim=512, backbone=bc_backbone)
  )

  model = PPO (
      "MlpPolicy",
      env,
      policy_kwargs=policy_kwargs,
      verbose=1,
      device="cpu"
  )

  TIMESTEPS = 50_000
  MAX_ITERS = 5
  iters = 0

  print("Training...")

  for iters in range(MAX_ITERS):
    print(f"Iter {iters}")

    model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False)
    #Se prefiere guardar los pesos del modelo ya que guardar el modelo directamente pueda dar problemas al cambiar de entorno
    model.save(f"{model_dir}/model_{iters}")
    #torch.save(model.)

In [75]:
#!ulimit -c unlimited

In [76]:
#!ulimit -c

In [77]:
#!cat /proc/sys/kernel/core_pattern

In [78]:
#!cat |/usr/lib/systemd/systemd-coredump

In [79]:
#!sudo sysctl -w kernel.core_pattern=core

In [80]:
#!ulimit -c

In [81]:
!file nKaruga

nKaruga: ELF 64-bit LSB pie executable, x86-64, version 1 (GNU/Linux), dynamically linked, interpreter /lib64/ld-linux-x86-64.so.2, BuildID[sha1]=4429aba3eb1f42561718dcf9e22c3a92a8c75335, for GNU/Linux 3.2.0, with debug_info, not stripped


In [82]:
!ls

 debug.log	 game_stderr.log  'model(1).pth.1'   nKaruga.1
 env_log.txt	 logs		   models	     sample_data
 game_data.csv	'model(1).pth'	   nKaruga


In [83]:
train()

ENV RESET CALLED
create server controller
server binding succesfull to ('127.0.0.1', 0)
[Listening] Server is listening on 127.0.0.1
SERVER STATE -> LISTENING
BOUND TO: ('127.0.0.1', 34777)
start headless
Starting game: ./nKaruga -C -V -S -P 34777
game process started
Using cpu device
Training...
Iter 0
ENV RESET CALLED
Logging to logs/PPO_0


KeyboardInterrupt: 

In [84]:
train_from_bc()

ENV RESET CALLED
create server controller
server binding succesfull to ('127.0.0.1', 0)
[Listening] Server is listening on 127.0.0.1
SERVER STATE -> LISTENING
BOUND TO: ('127.0.0.1', 45447)
start headless
Starting game: ./nKaruga -C -V -S -P 45447
game process started
Using cpu device
Training...
Iter 0
ENV RESET CALLED
TERMINATED
[ 4.770e+03  1.596e+04  0.000e+00  0.000e+00  0.000e+00  1.000e+02
  2.120e+02  0.000e+00 -1.000e+00  7.290e+02  4.860e+02  0.000e+00
  0.000e+00  1.500e+01  1.000e+01  0.000e+00  0.000e+00 -3.600e+02
  4.530e+02  0.000e+00  0.000e+00 -1.500e+01  1.000e+01  0.000e+00
  0.000e+00  2.370e+02  3.030e+02  0.000e+00  0.000e+00 -2.000e+00
  8.000e+00  0.000e+00  0.000e+00  8.100e+01  2.770e+02  0.000e+00
  1.000e+00 -3.000e+00  7.000e+00  0.000e+00  0.000e+00  2.260e+02
  2.530e+02  0.000e+00  0.000e+00  6.000e+00  7.000e+00  0.000e+00
  0.000e+00  1.170e+02  2.280e+02  0.000e+00  1.000e+00 -8.000e+00
  8.000e+00  0.000e+00  0.000e+00  1.790e+02  1.780e+02  0.000e+

KeyboardInterrupt: 

In [ ]:
#!cat env_log.txt

In [ ]:
!pip install torchinfo

In [ ]:
model = PPO.load("models/model_0")

In [ ]:
from torchinfo import summary
summary(model.policy)

In [ ]:
#!rm env_log.txt

In [ ]:
!unzip logs_20_000_steps.zip

In [ ]:
import os
from tensorboard import notebook

# Define the log directory based on your training function
log_dir = "logs/PPO_0"

# Ensure the log directory exists
if not os.path.exists(log_dir):
    print(f"Log directory '{log_dir}' not found. Please run the training first.")
else:
    # Load TensorBoard
    %load_ext tensorboard
    %tensorboard --logdir {log_dir}